<b>Scraping job listings from Indeed.com</b><br>

1st step is to check out the source of an Indeed.com page: (https://www.indeed.com/jobs?q=Data+Analyst&l=Washington%2C+DC&start=10). What we find is each listing is below a div tag with a class name of result. I am setting up a request (using requests) for the URL below and will use BeautifulSoup to parse the page and extract all results. The URL has the following query parameters<br>

- q for the job search<br>
- l for a location<br>
- start for what result number to start on

In [94]:
URL = "https://www.indeed.com/jobs?q=Data+Analyst&l=Washington%2C+DC&start=10"

From inspecting the data elements, we can see there is a structure for the details we need:<br>

- Salary is available in a nowrap element in a span with class='company'.<br>
- Job title is in a link with class='jobtitle' and a data-tn-element="jobTitle.<br>
- Location is set in a span with class='location'.<br>
- Company is set in a span with class='company'.<br>
- Job description is set in a span with class='summary'.

I am going to to write a function to extract these 5 items: location, company, job title, job description and salary.<br> I will be doing 2 things: checking if a field is empty or None for attempting to call methods on it and using try/except if I anticipate errors.<br> We will test it out with the URL above. 

In [95]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [96]:
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["title","location","company","salary", "summary"])
    for each in soup.find_all(class_= "result" ):
        try: 
            title = each.find(class_='jobtitle').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find('span', {'class':"location" }).text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='company').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find('span', {'class':'no-wrap'}).text
        except:
            salary = 'None'
        summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'title':title, 'location':location, 'company':company, 'salary':salary, 'summary':summary}, ignore_index=True)
    return df

In [97]:
parse(URL)

,title,location,company,salary,summary
0,DATA ANALYST/SPECIALIST - Admitting (Full Time),"Washington, DC 20037 (Foggy Bottom area)",The George Washington University Hospital,None,Previous experience with data extr...
1,Data Analyst,"Washington, DC 20003 (Capitol Hill area)",General Dynamics Information Technology,None,The Data Analyst supports a civili...
2,Data Analyst,"Washington, DC 20005 (Logan Circle area)",FON Advisors,None,The Data Analyst keeps an open min...
3,Junior Case/Data Analyst,"Washington, DC",IntelliWare Systems,None,The Junior Case/Data Analyst will ...
4,Domestic Law Enforcement Intelligence Analyst,"Washington, DC",ICF,None,"Geospatial and data analytics, for..."
5,Data Analyst,"Washington, DC","AnaVation, LLC",None,The Data Analyst shall develop two...
6,"Open Source Analyst, Junior","Alexandria, VA",Booz Allen Hamilton,None,"Open Source Analyst, Junior. Exper..."
7,Data Analyst I,"Alexandria, VA",InnovaSystems International,None,The Data Analyst will be responsib...
8,Sustainability Data Analyst,"Arlington, VA","Eastern Research Group, Inc.",None,"If so, our fast-paced consulting f..."
9,Data Analyst,"Washington, DC",ICF,None,Our staff provides deep technical ...


As we need more results, I am going to increase the scope to more cities and with max results per city of 150.<br>
There are two query parameters I will alter to collect more results, the l=Washington%2C+DC and the start=10.<br>The first controls the location of the results (we can pull multiple cities).<br>The second controls where in the results to start and gives 10 results (we can keep increasing by 10 to go further in the list).

In [98]:
url_template = "http://www.indeed.com/jobs?q=Data+Analyst+%2420%2C000&l={}&start={}"
max_results_per_city = 150 # Setting this to a small value(150) but more cities to generate more results. 

results = []
df_raw = pd.DataFrame(columns=["title","location","company","salary", "summary"])
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'San+Antonio', 
    'San+Diego', 'San+Jose', 'Detroit', 'Boston', 'Sacramento',
    'Arlington', 'New+Orleans', 'Pittsburgh', 'Washington%2C+DC']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = url_template.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text.replace('\n', '')
            except:
                salary = None
            try:
                summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                summary = None
            df_raw = df_raw.append({'title':title, 'location':location, 'company':company, 'salary':salary, 'summary':summary}, ignore_index=True)      

In [99]:
df_raw.to_csv('project_4_df_raw_not_cleaned.csv', encoding='utf-8')

In [209]:
df_raw = pd.read_csv('project_4_df_raw_not_cleaned.csv')

In [210]:
df_raw.drop('Unnamed: 0', axis=1, inplace=True)

In [211]:
df_raw.head()

,title,location,company,salary,summary
0,Production Data Analyst,"Houston, TX",Wunderman,NaN,"Data questions, service requests). Develop SQL..."
1,Listings Data Analyst - Houston,"Houston, TX 77056 (Galleria-Uptown area)",JLL,NaN,The Research Listings’ Data Analys...
2,Performance Apprentice,"Houston, TX 77002 (Downtown area)",Houston Astros,NaN,Manage athlete tracking technology...
3,Data Analyst I,"Houston, TX 77056 (Galleria-Uptown area)",Virtus Partners,NaN,Processes and reconciles data in v...
4,Data Analyst,"Houston, TX 77002 (Downtown area)",General Electric,NaN,"In the role of Data Analyst, you w..."


In [212]:
df_raw.tail()

,title,location,company,salary,summary
3587,Data Analyst-HEDIS & Star,"Long Beach, CA 90806",SCAN Health Plan,NaN,Data Analyst – HEDIS & Medicare Star. Document...
3588,Financial Analyst I - CA Dec 2018/May 2019 Grad,"El Segundo, CA",Raytheon,NaN,Experience in a financial setting ...
3589,"SQL Data Analyst, Analyst/Officer","Monterey Park, CA",MUFG,NaN,The Data Analyst works with Busine...
3590,NETWORK ANALYST (170244),"Los Angeles, CA",California State University,"$3,470 - $6,486 a month",The incumbent must have a strong u...
3591,Online Marketing Analyst,"Venice, CA",System1,NaN,Own the data analysis and visualiz...


I've used the function I wrote above to parse out the 5 fields - location, title, company, summary and salary and created a dataframe from the results with those 5 columns. Now, we need to clean up the salary data.<br>

As only a small number of scraped results have salary information - only these will be used for modeling.<br>
Some salaries are not yearly but hourly or weekly, these are not useful.<br>
Some entries may be duplicated.<br>
The salaries are given as text and with ranges.

In [213]:
df_raw.drop_duplicates(inplace=True) #dropping duplicates
df_raw.company.replace(regex=True,inplace=True,to_replace="\n",value="") #getting rid of /n in company
df_raw.salary.replace(regex=True, inplace=True, to_replace="\$", value="") #getting rid of $ in salary

In [214]:
def eda(dataframe): #function to check data info
    print ("missing values \n", dataframe.isnull().sum()) #shows total null values for each column
    print ("dataframe types \n", dataframe.dtypes)
    print ("dataframe shape \n", dataframe.shape)     
    print ("dataframe describe \n", dataframe.describe())
    print ("dataframe length =", len(dataframe)) #length of dataframe
    print ("duplicates", dataframe.duplicated().sum()) # added this to duplicates in data
    for item in dataframe:
        print (item)
        print (dataframe[item].nunique())

eda(df_raw)

missing values 
 title          0
location       0
company        1
salary      3001
summary        0
dtype: int64
dataframe types 
 title       object
location    object
company     object
salary      object
summary     object
dtype: object
dataframe shape 
 (3362, 5)
dataframe describe 
                title    location         company  \
count           3362        3362            3361   
unique          2423         779            1790   
top     Data Analyst  Austin, TX          Google   
freq             192         104              48   

                                        salary  \
count                                      361   
unique                                     314   
top                     45,541 - 59,203 a year   
freq                                         3   

                                                  summary  
count                                                3362  
unique                                               3182  
top              

In [215]:
print (df_raw.head())
print (df_raw.shape)
print (df_raw[df_raw.salary != 'None'].shape)
df_raw = df_raw[df_raw.salary != 'None'].drop_duplicates().dropna()
print (df_more.shape)

                             title                                  location  \
0          Production Data Analyst                               Houston, TX   
1  Listings Data Analyst - Houston  Houston, TX 77056 (Galleria-Uptown area)   
2           Performance Apprentice         Houston, TX 77002 (Downtown area)   
3                   Data Analyst I  Houston, TX 77056 (Galleria-Uptown area)   
4                     Data Analyst         Houston, TX 77002 (Downtown area)   

                    company salary  \
0                 Wunderman    NaN   
1                       JLL    NaN   
2            Houston Astros    NaN   
3           Virtus Partners    NaN   
4          General Electric    NaN   

                                             summary  
0  Data questions, service requests). Develop SQL...  
1              The Research Listings’ Data Analys...  
2              Manage athlete tracking technology...  
3              Processes and reconciles data in v...  
4              

In [216]:
df_raw = df_raw[df_raw.salary.str.contains("hour") == False]
df_raw = df_raw[df_raw.salary.str.contains("month") == False]
print (df_raw.shape)
df_raw.head()

(229, 5)


,title,location,company,salary,summary
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,"49,266 - 64,048 a year",Acquire data from primary or secon...
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","60,228 - 75,957 a year","Complies data, produces informatio..."
21,STAFF ANALYST,"Houston, TX","City of Houston, TX","56,602 - 87,698 a year","Performs research and support, com..."
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"98,103 a year","Designs, develops, and deploys dat..."
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,"46,030 a year",Special Education Data Analyst. Sp...


In [217]:
salaries = df_raw[df_raw.salary.notnull()]
salaries = salaries[salaries.salary.str.contains('year')] #getting yearly salaries
salaries.describe()

,title,location,company,salary,summary
count,227,227,227,227,227
unique,201,87,132,199,223
top,Program Analyst,"Washington, DC",University of Miami,"60,000 - 65,000 a year",Positions in the Inspector General...
freq,6,18,9,3,4


In [218]:
#splitting salaries to find an average
salaries.salary.replace(regex=True, inplace=True, to_replace="a year", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=",", value="")
salaries['salary_split'] = salaries['salary'].str.split('-')
salaries.head()

,title,location,company,salary,summary,salary_split
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,49266 - 64048,Acquire data from primary or secon...,"[ 49266 , 64048 ]"
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX",60228 - 75957,"Complies data, produces informatio...","[ 60228 , 75957 ]"
21,STAFF ANALYST,"Houston, TX","City of Houston, TX",56602 - 87698,"Performs research and support, com...","[ 56602 , 87698 ]"
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,98103,"Designs, develops, and deploys dat...",[ 98103 ]
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,46030,Special Education Data Analyst. Sp...,[ 46030 ]


In [219]:
#finding an average salary
def avg(salaries):
    salaries['lower'] = salaries['salary_split'].str[0].astype('float')
    salaries['upper'] = salaries['salary_split'].str[1].astype('float')
    salaries['avg'] = salaries[['lower','upper']].mean(axis=1)
avg(salaries)
salaries.head()

,title,location,company,salary,summary,salary_split,lower,upper,avg
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,49266 - 64048,Acquire data from primary or secon...,"[ 49266 , 64048 ]",49266.0,64048.0,56657.0
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX",60228 - 75957,"Complies data, produces informatio...","[ 60228 , 75957 ]",60228.0,75957.0,68092.5
21,STAFF ANALYST,"Houston, TX","City of Houston, TX",56602 - 87698,"Performs research and support, com...","[ 56602 , 87698 ]",56602.0,87698.0,72150.0
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,98103,"Designs, develops, and deploys dat...",[ 98103 ],98103.0,NaN,98103.0
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,46030,Special Education Data Analyst. Sp...,[ 46030 ],46030.0,NaN,46030.0


In [220]:
salaries[30:40]

,title,location,company,salary,summary,salary_split,lower,upper,avg
296,Analyst,"Manhattan, NY",New York City ADMIN FOR CHILDREN'S SVCS,50362 - 61280,Lead the development of performanc...,"[ 50362 , 61280 ]",50362.0,61280.0,55821.0
297,Analyst,"Manhattan, NY",NYC Administration for Children's Serv...,50362 - 61280,Lead the development of performanc...,"[ 50362 , 61280 ]",50362.0,61280.0,55821.0
298,Senior Data Analyst,"Queens, NY",New York City DEPT OF DESIGN & CONSTRU...,71330 - 111344,"Under direct supervision, with con...","[ 71330 , 111344 ]",71330.0,111344.0,91337.0
312,Quality Management Analyst 2 - Cardiovascular ...,"Coral Gables, FL",University of Miami,46900 - 75000,Required knowledge of the National...,"[ 46900 , 75000 ]",46900.0,75000.0,60950.0
340,Database Analyst - Part Time,"Coral Gables, FL",University of Miami,39300 - 62800,Extraction of data from UChart. De...,"[ 39300 , 62800 ]",39300.0,62800.0,51050.0
345,Cardiovascular Data Abstractor & Analyst,"Coral Gables, FL",University of Miami,56500 - 90000,Minimum 3 years’ experience with c...,"[ 56500 , 90000 ]",56500.0,90000.0,73250.0
357,GEOGRAPHIC INFORMATION SYSTEMS ANALYST,"Fort Lauderdale, FL","City of Fort Lauderdale, FL",63170 - 87006,It is your responsibility to provi...,"[ 63170 , 87006 ]",63170.0,87006.0,75088.0
372,HUMAN SERVICES PROGRAM ANALYST - 64031111,"Miami, FL 33125 (Flagami area)",The State of Florida,34000 - 36000,HUMAN SERVICES PROGRAM ANALYST. HU...,"[ 34000 , 36000 ]",34000.0,36000.0,35000.0
377,Quality Management Analyst 3,"Coral Gables, FL",University of Miami,56500 - 90400,Perform data retrieval for various...,"[ 56500 , 90400 ]",56500.0,90400.0,73450.0
384,Decision Support Analyst,"Coral Gables, FL",University of Miami,56500 - 90400,"Helps maintain, design and program...","[ 56500 , 90400 ]",56500.0,90400.0,73450.0


In [221]:
#dropping columns so I'm only left with lower, upper, avg salary columns
clean_df = salaries.drop(['salary','salary_split'], axis=1)
clean_df.head()

,title,location,company,summary,lower,upper,avg
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,Acquire data from primary or secon...,49266.0,64048.0,56657.0
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","Complies data, produces informatio...",60228.0,75957.0,68092.5
21,STAFF ANALYST,"Houston, TX","City of Houston, TX","Performs research and support, com...",56602.0,87698.0,72150.0
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"Designs, develops, and deploys dat...",98103.0,NaN,98103.0
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,Special Education Data Analyst. Sp...,46030.0,NaN,46030.0


In [222]:
clean_df = clean_df.join(clean_df['location'].str.split(',', 1, expand=True).rename(columns={0:'city', 1:'state'}))

In [223]:
clean_df.head()

,title,location,company,summary,lower,upper,avg,city,state
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,Acquire data from primary or secon...,49266.0,64048.0,56657.0,Houston,TX
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","Complies data, produces informatio...",60228.0,75957.0,68092.5,Houston,TX
21,STAFF ANALYST,"Houston, TX","City of Houston, TX","Performs research and support, com...",56602.0,87698.0,72150.0,Houston,TX
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"Designs, develops, and deploys dat...",98103.0,NaN,98103.0,Houston,TX
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,Special Education Data Analyst. Sp...,46030.0,NaN,46030.0,Houston,TX


In [224]:
def strip_state(x):
    if x != None:
        return x[0:3]
    else:
        None
clean_df['state initials'] = clean_df['state'].apply(strip_state)
clean_df.head()

,title,location,company,summary,lower,upper,avg,city,state,state initials
9,Project Data Analyst,"Houston, TX",Baylor College of Medicine,Acquire data from primary or secon...,49266.0,64048.0,56657.0,Houston,TX,TX
15,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","Complies data, produces informatio...",60228.0,75957.0,68092.5,Houston,TX,TX
21,STAFF ANALYST,"Houston, TX","City of Houston, TX","Performs research and support, com...",56602.0,87698.0,72150.0,Houston,TX,TX
32,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"Designs, develops, and deploys dat...",98103.0,NaN,98103.0,Houston,TX,TX
33,Special Education Data Analyst,"Houston, TX",Houston Independent School District,Special Education Data Analyst. Sp...,46030.0,NaN,46030.0,Houston,TX,TX


<b>Saving my cleaned results to CSV.</b>

In [225]:
clean_df.to_csv('project_4_cleaned.csv', encoding='utf-8')

Now, I'm going attempt to predict salaries using Random Forests and other models, where possible.<br>
First, loading in the cleaned data of scraped salaries.

In [226]:
clean_df = pd.read_csv('project_4_cleaned.csv', index_col=0)
clean_df = clean_df.reset_index(drop=True)
clean_df.head()

,title,location,company,summary,lower,upper,avg,city,state,state initials
0,Project Data Analyst,"Houston, TX",Baylor College of Medicine,Acquire data from primary or secon...,49266.0,64048.0,56657.0,Houston,TX,TX
1,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","Complies data, produces informatio...",60228.0,75957.0,68092.5,Houston,TX,TX
2,STAFF ANALYST,"Houston, TX","City of Houston, TX","Performs research and support, com...",56602.0,87698.0,72150.0,Houston,TX,TX
3,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"Designs, develops, and deploys dat...",98103.0,NaN,98103.0,Houston,TX,TX
4,Special Education Data Analyst,"Houston, TX",Houston Independent School District,Special Education Data Analyst. Sp...,46030.0,NaN,46030.0,Houston,TX,TX


The aim is to predict a binary variable - whether salary was high or low. I am going to find the median salary and create a new binary variable that is true when the salary is above the median. From reading, I see we can use Linear Regression to predict salary value but this is going to be a binary classification problem - predicting 2 classes: High vs Low salary. There is also the option of not using median as the splitting point, ie we can split on 75th percentile or other points but keeping it simple for now. 

In [228]:
median = clean_df['avg'].median()
print ('The median salary for our data set is $' + str(median))

The median salary for our data set is $62292.0


In [229]:
def above_median(x):
    if x > median:
        return 1
    return 0

clean_df['above median'] = clean_df['avg'].apply(above_median)
clean_df.head()

,title,location,company,summary,lower,upper,avg,city,state,state initials,above median
0,Project Data Analyst,"Houston, TX",Baylor College of Medicine,Acquire data from primary or secon...,49266.0,64048.0,56657.0,Houston,TX,TX,0
1,STAFF ANALYST/DO,"Houston, TX","City of Houston, TX","Complies data, produces informatio...",60228.0,75957.0,68092.5,Houston,TX,TX,1
2,STAFF ANALYST,"Houston, TX","City of Houston, TX","Performs research and support, com...",56602.0,87698.0,72150.0,Houston,TX,TX,1
3,Data Warehouse Analyst,"Houston, TX",Baylor College of Medicine,"Designs, develops, and deploys dat...",98103.0,NaN,98103.0,Houston,TX,TX,1
4,Special Education Data Analyst,"Houston, TX",Houston Independent School District,Special Education Data Analyst. Sp...,46030.0,NaN,46030.0,Houston,TX,TX,0


Attempting to see what the baseline accuracy of this model will be:

In [230]:
clean_df['above median'].value_counts()

0    114
1    113
Name: above median, dtype: int64

The baseline accuracy will be about 50/50 as we have a 50/50 chance of whether or not a value is above or below the median.

I am going to create a Random Forest model to predict High/Low salary using Sklearn. I will be starting with only using <b>location</b> as a feature.

In [233]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from ipywidgets import *
from IPython.display import display
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split

In [234]:
df_city_dummy = pd.get_dummies(clean_df['city'])
df_city_dummy.head()

,Albany,Alhambra,Arlington,Atlanta,Austin,Bellevue,Bethpage,Boston,Bremerton,Brighton,...,Seattle,Staten Island,Sunnyvale,Tacoma,Tempe,Tukwila,Vallejo,Washington,Waterton,Woodland
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [235]:
df_state_dummy = pd.get_dummies(clean_df['state initials'])
df_state_dummy.head()

,AZ,CA,CO,DC,FL,GA,IL,LA,MA,MI,NY,OR,PA,TX,VA,WA
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [237]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
X = df_city_dummy
y = clean_df['above median']
cv_model = cross_val_score(model, X, y, cv=6)
print ('Cross-validated scores:', cv_model)
print ('Average score:', cv_model.mean())
print ('Standard deviation of score:', cv_model.std())
model.fit(X, y)
print (model.oob_score_)

Cross-validated scores: [0.36842105 0.28947368 0.57894737 0.5        0.55263158 0.62162162]
Average score: 0.4851825509720247
Standard deviation of score: 0.1184001669377106
0.5991189427312775


It seems like city locations are only slightly better than our baseline accuracy of 50/50.

What I am going to do now is create a few new variables to show interesting features of a job title. ie. create a feature that shows if 'Senior' or 'Manager' is in the job title, create a new Random Forest model with these new features and see if they add any value. After which, I will attempt to use an NLP technique: count-vectorizer to create features based on the words in the job titles. Another Random Forest model will be created with both location and these new features.

In [238]:
def senior(x):
    if 'Senior' in x:
        return 1
    return 0

clean_df['senior'] = clean_df['title'].apply(senior)

In [239]:
clean_df[clean_df.senior != 0]

,title,location,company,summary,lower,upper,avg,city,state,state initials,above median,senior
5,Senior Sponsored Programs Analyst,"Houston, TX",Baylor College of Medicine,Prepares reports using BRAIN data ...,54685.0,67291.0,60988.0,Houston,TX,TX,0,1
32,Senior Data Analyst,"Queens, NY",New York City DEPT OF DESIGN & CONSTRU...,"Under direct supervision, with con...",71330.0,111344.0,91337.0,Queens,NY,NY,1,1
48,Senior Performance Measurement Analyst,"Detroit, MI",Henry Ford Health System,Explores and recommends measuremen...,72218.0,NaN,72218.0,Detroit,MI,MI,1,1
49,Senior Systems Analyst,"Detroit, MI",Henry Ford Health System,Develops plans for the implementat...,59675.0,NaN,59675.0,Detroit,MI,MI,0,1
68,Senior Management Analyst (Director's Office),"San Antonio, TX 78205 (Downtown area)",City of San Antonio,Describe your experience in utiliz...,52683.0,79024.0,65853.5,San Antonio,TX 78205 (Downtown area),TX,1,1
74,Compliance Senior Analyst,"San Antonio, TX 78205 (Downtown area)",City of San Antonio,COMPLIANCE SENIOR ANALYST. Complia...,47894.0,71840.0,59867.0,San Antonio,TX 78205 (Downtown area),TX,0,1
75,Senior Time & Labor Analyst,"San Antonio, TX",University of Texas at San Antonio,B) Ability to analyze statistical ...,37812.0,62388.0,50100.0,San Antonio,TX,TX,0,1
76,Senior Budget Analyst,"San Antonio, TX",University of Texas at San Antonio,Review and process documents for a...,47016.0,77568.0,62292.0,San Antonio,TX,TX,0,1
81,"Data/Reporting Analyst, Senior","Atlanta, GA","City of Atlanta, GA","Data/Reporting Analyst, Senior. Position does ...",41000.0,NaN,41000.0,Atlanta,GA,GA,0,1
94,"Budget Analyst I, II, III or Senior","Austin, TX",Travis County,"Evaluates data from national, stat...",47476.0,88525.0,68000.5,Austin,TX,TX,1,1


In [240]:
def manager(x):
    if 'Manager' in x:
        return 1
    return 0

clean_df['manager'] = clean_df['title'].apply(manager)

In [241]:
clean_df[clean_df.manager != 0]

,title,location,company,summary,lower,upper,avg,city,state,state initials,above median,senior,manager


In [242]:
def engineer(x):
    if 'Engineer' in x:
        return 1
    return 0

clean_df['engineer'] = clean_df['title'].apply(engineer)

In [243]:
clean_df[clean_df.engineer != 0]

,title,location,company,summary,lower,upper,avg,city,state,state initials,above median,senior,manager,engineer


In [244]:
def data_scientist(x):
    if 'Data Scientist' in x:
        return 1
    return 0

clean_df['data Scientist'] = clean_df['title'].apply(data_scientist)

In [245]:
def analyst(x):
    if 'Analyst' in x:
        return 1
    return 0

clean_df['analyst'] = clean_df['title'].apply(analyst)

In [246]:
def assistant(x):
    if 'Assistant' in x:
        return 1
    return 0

clean_df['assistant'] = clean_df['title'].apply(assistant)

In [247]:
def internship(x):
    if 'Intern' in x:
        return 1
    return 0

clean_df['intern'] = clean_df['title'].apply(internship)

In [248]:
def developer(x):
    if 'Developer' in x:
        return 1
    return 0

clean_df['developer'] = clean_df['title'].apply(developer)

In [249]:
def machine_learning(x):
    if 'Machine Learning' in x:
        return 1
    return 0

clean_df['machine learning'] = clean_df['title'].apply(machine_learning)

In [251]:
feature_matrix = clean_df.copy(deep=True)
feature_matrix.drop(['title', 'location', 'company', 'summary', 'lower', 'upper', 'avg', 'city', 'state', 'above median', 'state initials'], axis=1, inplace=True)
print (feature_matrix.shape)
feature_matrix.head()

(227, 9)


,senior,manager,engineer,data Scientist,analyst,assistant,intern,developer,machine learning
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0


In [253]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
X_features = pd.concat([df_city_dummy, feature_matrix, df_state_dummy], axis=1)
y = clean_df['above median']
cv_model = cross_val_score(model, X_features, y, cv=6)
print ('Cross-validated scores:', cv_model)
print ('Average score:', cv_model.mean())
print ('Standard deviation of score:', cv_model.std())
model.fit(X_features, y)
print (model.oob_score_)

Cross-validated scores: [0.44736842 0.42105263 0.44736842 0.78947368 0.65789474 0.7027027 ]
Average score: 0.5776434329065908
Standard deviation of score: 0.1445775366370974
0.6343612334801763


Having added the keywords ('Senior', 'Manager', 'Data Scientist', 'Analyst', 'Assistant', 'Intern', and 'Developer), the model accuracy was brought up to around 63% from 50%. Now, as mentioned above, I am going to attempt to increase this by using a count-vectorizer on the job title column.

In [254]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(stop_words='english', max_features=30, ngram_range=(2,2))
vectorizers = cvec.fit_transform(clean_df['title'].values)

df_vec  = pd.DataFrame(vectorizers.todense(), columns=cvec.get_feature_names())
print (df_vec.shape)
df_vec.head()

(227, 30)


,analyst ii,analyst open,analyst police,analyst real,analyst senior,budget analyst,business analyst,business intelligence,business systems,data analyst,...,program analyst,project planning,reporting analyst,research analyst,resources analyst,senior analyst,services analyst,staff analyst,support analyst,systems analyst
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [255]:
X_cvec = pd.concat([df_vec, df_city_dummy, df_state_dummy], axis=1)
X_cvec.head()

,analyst ii,analyst open,analyst police,analyst real,analyst senior,budget analyst,business analyst,business intelligence,business systems,data analyst,...,IL,LA,MA,MI,NY,OR,PA,TX,VA,WA
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [257]:
model = RandomForestClassifier(n_estimators=100, oob_score=True)
y = clean_df['above median']
cv_model = cross_val_score(model, X_cvec, y, cv=6)
print ('Cross-validated scores:', cv_model)
print ('Average score:', cv_model.mean())
print ('Standard deviation of score:', cv_model.std())
model.fit(X_cvec, y)
print (model.oob_score_)
importance_dataframe = pd.DataFrame(model.feature_importances_, index = X_cvec.columns, columns=['importance']).sort_values('importance', ascending=False)
importance_dataframe.head(20)

Cross-validated scores: [0.55263158 0.47368421 0.42105263 0.73684211 0.55263158 0.67567568]
Average score: 0.5687529634898056
Standard deviation of score: 0.10882683483331484
0.6255506607929515


,importance
data analyst,0.078427
CA,0.059095
research analyst,0.032100
analyst ii,0.030898
program analyst,0.027956
management analyst,0.024231
AZ,0.023933
Phoenix,0.019524
Pittsburgh,0.019383
NY,0.019190


The top description features are data analyst, CA, research analyst, program analyst and management analyst.<br>Using the count vectorizer (and a max of 30 new word features) though has slighlty decreased our accuracy to around 62%.

Now, I am going to use cross-validation in scikit-learn to evaluate the accuracy of the model above. The Random Forest Classifier with the description words count-vectorized and the location dummies has an average accuracy score of 62%.

Repeating the model-building process with a non tree-based model.

In [258]:
from sklearn import linear_model
log_reg = linear_model.LogisticRegression()
scores_log = cross_val_score(log_reg, X_cvec, y, cv=6)
print ('Cross-validated scores:', scores_log)
print ('Average score:', scores_log.mean())
print ('Standard deviation of score:', scores_log.std())
log_model = log_reg.fit(X_cvec, y)

/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Cross-validated scores: [0.5        0.47368421 0.44736842 0.73684211 0.5        0.59459459]
Average score: 0.54208155523945
Standard deviation of score: 0.09821725941494153


/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The Logistic Regression model has an accuracy score of 54%, which is poor against the Random Forest Classifier of 62%.

In [259]:
from sklearn.svm import SVC
model_svmrbf = SVC(kernel='rbf')
scores_svm = cross_val_score(model_svmrbf, X_cvec, y, cv=6)
print ('Cross-validated scores:', scores_svm)
print ('Average score:', scores_svm.mean())
print ('Standard deviation of score:', scores_svm.std())
svm_model = model_svmrbf.fit(X_cvec, y)

Cross-validated scores: [0.5        0.5        0.5        0.5        0.5        0.59459459]
Average score: 0.5157657657657658
Standard deviation of score: 0.03525332396959129


/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/shaleniiedevi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/shaleniiedevi/anaconda3/

In [260]:
model_svmlm = SVC(kernel='linear')
scores_svmlm = cross_val_score(model_svmlm, X_cvec, y, cv=6)
print ('Cross-validated scores:', scores_svmlm)
print ('Average score:', scores_svmlm.mean())
print ('Standard deviation of score:', scores_svmlm.std())
svm_model = model_svmlm.fit(X_cvec, y)

Cross-validated scores: [0.47368421 0.44736842 0.52631579 0.71052632 0.55263158 0.62162162]
Average score: 0.5553579895685159
Standard deviation of score: 0.08911725813741812


The linear kernel support vector machine performs better than the radial kernel support vector machine with an accuracy score of 55% which is still poorer than the Random Forest Classifier.

In [262]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_cvec, y)
cv_model = cross_val_score(clf, X_cvec, y, cv=6)
print ('Cross-validated scores:', cv_model)
print ('Average score:', cv_model.mean())
print ('Standard deviation of score:', cv_model.std())

Cross-validated scores: [0.52631579 0.42105263 0.5        0.65789474 0.55263158 0.67567568]
Average score: 0.5555950687529635
Standard deviation of score: 0.08845254074680219


The Gradient Boosting Classifier with an accuracy score of 55% is still poorer than the Random Forest Classifier.

End Notes: While my base model gave a decent accuracy of 62%, all the cross validation models have performed poorer against it(Logisitc Regression, Radial Kernel SVM, Linear Kernel SVM and Gradient Boosting Classifier). My assumption is that the small sample size of 227 salary observations is hurting the models. The next steps will be to review the data cleaning steps again and draw out a much larger sample size of 500-1000 to work with. 